In [1]:
import math

In [2]:
import gurobipy as gp

In [3]:
from gurobipy import GRB

**Setup**

In [4]:
height = 10
width = 10
spaces = height * width

In [5]:
#set snakes
inputSnakePairs = [[3, 37], [6, 16], [15, 9], [49, 12], [14, 32],
  [27, 56], [61, 22], [42, 17], [88, 36], [39, 44],
  [58, 45], [75, 47], [94, 64], [97, 65], [69, 87],
  [79, 98], [41, 85], [89, 91]]
snakePairs = []
for [head, tail] in inputSnakePairs:
    if head < spaces - 1 and tail < spaces - 1:
        snakePairs.append([head-1, tail-1])
snakes = len(snakePairs)

In [6]:
import random

In [7]:
def normalBoard():
    result = [];
    for space in range(spaces):
        j = space % width
        i = int(space / width)
        if (i%2) == 1:
            j = (width-1) - j;
        result = result + [(height-1 - i) + j * width]
    return result

def shuffledBoard():
    result = normalBoard()
    for v in range(spaces-2):
        u = random.randrange(v+2, spaces, 2)
        [result[u], result[v]] = [result[v], result[u]]
    return result

In [8]:
#board[space] = square.  Space is the position of the tour, square is the location on the board
def getLocation(b, space):
    square = b[space]
    return [int(square/width), square%width]   # row, col
            
def getSpace(b, i, j):
    square = i + j * width
    return b.index(square)

def displayBoard(b):
    print("----------------------")
    print(cost(b))
    print()
    for i in range(height):
        output = ""
        for j in range(width):
            output += str(getSpace(b, i, j)+1).ljust(4)
        print(output)
    print("----------------------")

In [9]:
def distance(b, space0, space1):
    [i0, j0] = getLocation(b, space0)
    [i1, j1] = getLocation(b, space1)
    return math.hypot(i0 - i1, j0 - j1)

def cost(b):
    neighbor_sum = 0
    for s in range(spaces-1):
        neighbor_sum = neighbor_sum + distance(b, s, s+1) - 1
    snake_sum = 0
    for snake in range(snakes):
        [head, tail] = snakePairs[snake]
        snake_sum = snake_sum - distance(b, head, tail)
        
    return snake_sum + neighbor_sum * 1000

**Board, path constraints**

In [10]:
def reverse_segment_if_better3(tour, i, j, k):
    cost0 = cost(tour)
    
    tourA = tour.copy()
    tourA[i:j] = reversed(tourA[i:j])
    costA = cost(tourA)
    
    tourB = tour.copy()
    tourB[j:k] = reversed(tourB[j:k])
    costB = cost(tourB)    
    
    tourC = tour.copy()
    tourC[i:k] = reversed(tourC[i:k])
    costC = cost(tourC)    
    
    tourD = tour.copy()
    tmp = tourD[j:k] + tourD[i:j]
    tourD[i:k] = tmp
    costD = cost(tourD)
    
    if cost0 < min(costA, costB, costC, costD):
        return tour
    elif costA < min(costB, costC, costD):
        return tourA
    elif costB < min(costC, costD):
        return tourB 
    elif costC < costD:
        return tourC
    else:
        return tourD
    
def reverse_segment_if_better2(tour, i, j):
    cost0 = cost(tour)
    
    tourA = tour.copy()
    tourA[i:j] = reversed(tourA[i:j])
    costA = cost(tourA)

    
    if cost0 < costA:
        return tour
    else:
        return tourA

In [11]:
def three_opt(tour):
    """Iterative improvement based on 3 exchange."""
    newcost = cost(tour)
    currentcost = newcost+1
    while(newcost < currentcost):
        for step in range(30000):
            i = random.randrange(spaces-6)
            j = random.randrange(i+3, spaces-3, 2)
            k = random.randrange(j+3, spaces + (i>0), 2)
            tour = reverse_segment_if_better3(tour, i, j, k)
        currentcost = newcost;    
        newcost = cost(tour)
    return tour

def two_opt(tour):
    """Iterative improvement based on 2 exchange."""
    newcost = cost(tour)
    currentcost = newcost+1
    while(newcost < currentcost):
        for step in range(5000):
            i = random.randrange(spaces-3)
            j = random.randrange(i+3, spaces, 2)
            tour = reverse_segment_if_better2(tour, i, j)
        currentcost = newcost;    
        newcost = cost(tour)
    return tour

In [12]:
board = normalBoard()
# board = shuffledBoard()

In [ ]:
bestScore = cost(board)
displayBoard(board)
while True:
    board = two_opt(shuffledBoard())
    while cost(board) > 1000: 
        board = two_opt(shuffledBoard())
    board = three_opt(board)
    print("   " + str(cost(board)))
    if cost(board) < bestScore:
        bestScore = cost(board)
        displayBoard(board)

----------------------
-104.21070112481273

54  55  56  57  58  59  60  63  64  65  
53  52  87  86  83  82  61  62  67  66  
50  51  88  85  84  81  74  73  68  69  
49  90  89  98  97  80  75  72  71  70  
48  91  100 99  96  79  76  19  18  17  
47  92  93  94  95  78  77  20  15  16  
46  45  44  29  28  25  24  21  14  13  
41  42  43  30  27  26  23  22  11  12  
40  37  36  31  32  7   8   9   10  1   
39  38  35  34  33  6   5   4   3   2   
----------------------
   -106.03471264371245
----------------------
-106.03471264371245

38  39  46  47  56  57  58  59  60  61  
37  40  45  48  55  54  65  64  63  62  
36  41  44  49  52  53  66  67  76  77  
35  42  43  50  51  100 69  68  75  78  
34  33  94  95  98  99  70  71  74  79  
31  32  93  96  97  86  85  72  73  80  
30  29  92  91  90  87  84  83  82  81  
1   28  27  26  89  88  21  20  17  16  
2   5   6   25  24  23  22  19  18  15  
3   4   7   8   9   10  11  12  13  14  
----------------------
   -108.39927667443521


**Scratch space**

In [ ]:
cost(b)